#Prerequistes

In [ ]:
!python --version

Python 3.7.13


##Libraries

In [ ]:
!pip install imageio==2.4.1
!pip install youtube_dl
!pip install opencv-python
!pip install keras
!pip install DeepFace
!pip install retina-face
!pip install cvlib
!pip install youtube_dl
!pip install facenet_pytorch
!pip install mtcnn
!pip install pytube

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.3 MB 9.7 MB/s 
  Created wheel for imageio: filename=imageio-2.4.1-py3-none-any.whl size=3303885 sha256=48e1bd97adac6a2b885be056d1b9fa8ace58eeee7435fd8029024c5f7f517107
  Stored in directory: /root/.cache/pip/wheels/46/20/07/7bb9c8c44e6ec2efa60fd0e6280094f53f65f41767ef69a5ee
Successfully built imageio
  Attempting uninstall: imageio
    Found existing installation: imageio 2.9.0
    Uninstalling imageio-2.9.0:
      Successfully uninstalled imageio-2.9.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.9 MB 6.7 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, h

##Mount drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Face Detection and Gender Prediction

In [ ]:
 # Import Libraries
import cv2
import re
import numpy as np
import os
from moviepy.editor import *
import youtube_dl
import pandas as pd
from retinaface import RetinaFace
import cvlib as cv
from google.colab.patches import cv2_imshow
from deepface import DeepFace as dp
import numpy as np
import time
from pytube import YouTube
import pandas as pd

In [ ]:
start_time = time.time()

def gender_predict(video):
    '''Predict the gender in a video'''

    male_fps = 0
    female_fps = 0
    total_fps = 0
    global duration
    global fps
    global frame_count
    global count
    count = 0

    try:
      # create a new cam object
      cap = cv2.VideoCapture(video)
      fps = cap.get(cv2.CAP_PROP_FPS)
      frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
      duration = frame_count/fps
      print(f'Duration:{duration:.3f}, FPS:{int(fps)}, Total Frames:{frame_count}')
      
      while True:
          
          # total frames
          total_fps += 1
          
          # read frame from video 
          status, frame = cap.read()

          # predict the faces
          # start_time2 = time.time()
          obj = RetinaFace.detect_faces(frame)

          male_flag, female_flag = 0, 0
          
          if type(obj) != dict:
              cv2.imwrite('/content/drive/MyDrive/YouTube-Gender-Prediction-Using-Faces-main/Data/Gender Detection/video_frames/'+str(total_fps)+'.jpg',frame)
          else:
              count += 1
              # Loop over the faces detected
              for key in obj.keys():
                  identity = obj[key]
                  
                  face_area = identity["facial_area"]
                  cv2.imwrite('/content/drive/MyDrive/YouTube-Gender-Prediction-Using-Faces-main/Data/Gender Detection/video_frames/0.jpg', cv2.resize(frame[face_area[1]:face_area[3], face_area[0]:face_area[2]], (224,224)))
                  dpa = dp.analyze('/content/drive/MyDrive/YouTube-Gender-Prediction-Using-Faces-main/Data/Gender Detection/video_frames/0.jpg', actions = ["gender"],enforce_detection=False, detector_backend = 'retinaface')
                  label = dpa['gender']

                  if label == 'Man':
                      male_flag = 1
                  if label == 'Woman': 
                      female_flag = 1

                  box_color = (255, 0, 0) if label == "Man" else (147, 20, 255)
                  cv2.rectangle(frame, (face_area[2], face_area[3]), (face_area[0], face_area[1]), box_color, 1)
                  
                  # Label processed image
                  cv2.putText(frame, label, (face_area[0],face_area[1]),
                              cv2.FONT_HERSHEY_SIMPLEX, 0.7, box_color, 2)
              

              if male_flag == 1:
                male_fps += 1
              if female_flag == 1:
                female_fps += 1
              # Save the processed image
              cv2.imwrite('/content/drive/MyDrive/YouTube-Gender-Prediction-Using-Faces-main/Data/Gender Detection/video_frames/'+str(total_fps)+'.jpg',frame)
    
    except: Exception
    non_human_fps = 0
    print(f'Count = {count}')
    if male_fps + female_fps <= total_fps:
        non_human_fps = total_fps - (male_fps + female_fps)
        print(f'Total Duration: {duration:.3f}\nMale Screen Time: {male_fps/fps:.3f}\nFemale Screen Time:{female_fps/fps:.3f}\nTotal Non-Human Time: {non_human_fps/fps:.3f}')
    
    
    col1 = "Video"
    col6 = "Duration"
    col2 = "Male Screen Time"
    col3 = "Female Screen Time"
    col4 = "Non-Humqn Screen Time"
    col5 = "Views"
    data = pd.DataFrame({col1:title,col6:duration,col2:male_fps/fps,col3:female_fps/fps,col4:non_human_fps/fps,col5:views})
    data.to_excel('YouTube_Stats.xlsx', sheet_name='sheet1', index=False)
    
    return ""

def alpha_num(text):
    return str(re.sub(r'[^A-Za-z0-9 ]+', '', text))

def atoi(text):
    '''Return numerical values'''
    return int(text) if text.isdigit() else text

def natural_keys(text):
    '''Return list of values'''
    return [ atoi(c) for c in re.split(r'(\d+)', text) ]

def remove_frames():
    '''Delete frames'''
    # Delete frames
    dir = '/content/drive/MyDrive/YouTube-Gender-Prediction-Using-Faces-main/Data/Gender Detection/video_frames'
    for f in os.listdir(dir):
        os.remove(os.path.join(dir, f))

def img_to_vid():
    '''Convert sequence of frames into MP4 video'''

    img_array = []

    # Make an array of frames to be combined
    for filename in os.listdir('/content/drive/MyDrive/YouTube-Gender-Prediction-Using-Faces-main/Data/Gender Detection/video_frames/'):
        if filename != '0.jpg':
            img_array.append(os.path.join('/content/drive/MyDrive/YouTube-Gender-Prediction-Using-Faces-main/Data/Gender Detection/video_frames/',filename))
    
    # Sort the frames in order
    img_array.sort(key=natural_keys)

    # Combine to form MP4 with required fps
    clip = ImageSequenceClip(img_array, fps=fps) 
    clip.write_videofile("/content/drive/MyDrive/YouTube-Gender-Prediction-Using-Faces-main/Data/Gender Detection/processed_video/"+title+".mp4", fps=int(fps))

    remove_frames()

if __name__ == '__main__':
    
    global title
    global views
    # https://www.youtube.com/watch?v=5hEeOtYYVdQ
    url = input("Enter URL: ")
    
    video = YouTube(url)
    title = video.title
    views = video.views

    path = video.streams.filter(file_extension = "mp4").order_by('resolution')[-1].download('/content/drive/MyDrive/YouTube-Gender-Prediction-Using-Faces-main/Data/Gender Detection/video')
    print(path)
    #path = '/content/drive/MyDrive/YouTube-Gender-Prediction-Using-Faces-main/Data/Gender Detection/Make It Extraordinary Albert Bartlett 10 Sec TV Ad 2021.mp4'
    gender_predict(path)
    img_to_vid()

    end_time = time.time()
    print(f"Execution time: {end_time-start_time}")  #86 mins for a min long video

Enter URL: https://www.youtube.com/watch?v=5hEeOtYYVdQ
/content/drive/MyDrive/YouTube-Gender-Prediction-Using-Faces-main/Data/Gender Detection/video/BBC Minute News in 60 seconds - BBC News.mp4
Duration:60.680, FPS:50, Total Frames:3034
retinaface.h5 will be downloaded from the url https://github.com/serengil/deepface_models/releases/download/v1.0/retinaface.h5


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/retinaface.h5
To: /root/.deepface/weights/retinaface.h5
100%|██████████| 119M/119M [00:13<00:00, 8.81MB/s]


gender_model_weights.h5 will be downloaded...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/gender_model_weights.h5
To: /root/.deepface/weights/gender_model_weights.h5
100%|██████████| 537M/537M [00:52<00:00, 10.3MB/s]


Count = 1599
Total Duration: 60.680
Male Screen Time: 29.480
Female Screen Time:13.680
Total Non-Human Time: 17.540
[MoviePy] >>>> Building video /content/drive/MyDrive/YouTube-Gender-Prediction-Using-Faces-main/Data/Gender Detection/processed_video/ex.mp4
[MoviePy] Writing video /content/drive/MyDrive/YouTube-Gender-Prediction-Using-Faces-main/Data/Gender Detection/processed_video/ex.mp4


100%|██████████| 3035/3035 [01:29<00:00, 33.74it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: /content/drive/MyDrive/YouTube-Gender-Prediction-Using-Faces-main/Data/Gender Detection/processed_video/ex.mp4 

Execution time: 1685.2450551986694


#FACE DETECTION

##DLIB

In [ ]:
!pip install face-recognition

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 100.1 MB 20 kB/s 
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566186 sha256=5ea9ea7a8ebe3a4f29601fee9333d601d1bdb605026636bca661cb9f46ccfe35
  Stored in directory: /root/.cache/pip/wheels/d6/81/3c/884bcd5e1c120ff548d57c2ecc9ebf3281c9a6f7c0e7e7947a
Successfully built face-recognition-models


In [ ]:
import dlib
import cv2
import time

start_time = time.time()
#We create the model here with the weights placed as parameters
face_detect = dlib.cnn_face_detection_model_v1("/content/drive/MyDrive/YouTube-Gender-Prediction-Using-Faces-main/Data/Gender Detection/weights/mmod_human_face_detector.dat")

cap = cv2.VideoCapture("/content/drive/MyDrive/YouTube-Gender-Prediction-Using-Faces-main/Data/Gender Detection/Make It Extraordinary Albert Bartlett 10 Sec TV Ad 2021.mp4")
count = 0
#try:
while True:
    ret, frame = cap.read()
    frame = cv2.resize(frame, (600, 400))

    faces = face_detect(frame, )
        
    for face in faces:
        count += 1
        # In dlib in order to extract points we need to do this
        x1 = face.rect.left()
        y1 = face.rect.bottom()
        x2 = face.rect.right()
        y2 = face.rect.top()
        cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 1)
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break
#except: Exception
   
print(count)
cap.release()
cv2.destroyAllWindows()
end_time = time.time()
print(f"Execution time: {end_time-start_time}") 

In [ ]:
import dlib
import cv2
import time

detector = dlib.cnn_face_detection_model_v1("/content/drive/MyDrive/YouTube-Gender-Prediction-Using-Faces-main/Data/Gender Detection/weights/mmod_human_face_detector.dat")
start_time = time.time()

path = '/content/drive/MyDrive/YouTube-Gender-Prediction-Using-Faces-main/Data/Gender Detection/test_images/img0.jpg'
img = cv2.imread(path)

faces = detector(img, 1)
    
for face in faces:
    count += 1
    # In dlib in order to extract points we need to do this
    x1 = face.rect.left()
    y1 = face.rect.bottom()
    x2 = face.rect.right()
    y2 = face.rect.top()
    cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 1)

cv2.imwrite('/content/drive/MyDrive/YouTube-Gender-Prediction-Using-Faces-main/Data/Gender Detection/test_images/img0(dlib).jpg',img)

print(count)
#cap.release()
end_time = time.time()
print(f"Execution time: {end_time-start_time}") 

##Facenet

In [ ]:
from facenet_pytorch import MTCNN
import torch
import cv2
import time

start_time = time.time()
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
 
#Create the model
mtcnn = MTCNN(keep_all=True, device=device)
 
#Load the video and go from frame to frame
cap = cv2.VideoCapture("/content/drive/MyDrive/YouTube-Gender-Prediction-Using-Faces-main/Data/Gender Detection/Make It Extraordinary Albert Bartlett 10 Sec TV Ad 2021.mp4")
count = 0
while True:
   ret, frame = cap.read()
   if ret:
       frame = cv2.resize(frame, (600, 400))

      #Here we are going to use the facenet detector
       boxes, conf = mtcnn.detect(frame)

      # If there is no confidence that in the frame is a face, don't draw a rectangle around it
       if conf[0] !=  None:     
            for (x, y, w, h) in boxes:
                count += 1
                text = f"{conf[0]*100:.2f}%"
                x, y, w, h = int(x), int(y), int(w), int(h)
                cv2.rectangle(frame, (x, y), (w, h), (255, 255, 255), 1)
   else:
       break

#Show the result
#If we were using Google Colab we would use their function cv2_imshow()

#For displaying images/frames
   if cv2.waitKey(25) & 0xFF == ord('q'):
       break

print(count)
cap.release()
cv2.destroyAllWindows()
end_time = time.time()
print(f"Execution time: {end_time-start_time}") 

164
Execution time: 7.263085126876831


In [ ]:
from facenet_pytorch import MTCNN
import torch
import cv2
import time

start_time = time.time()
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
 
#Create the model
mtcnn = MTCNN(keep_all=True, device=device)

path = '/content/drive/MyDrive/YouTube-Gender-Prediction-Using-Faces-main/Data/Gender Detection/test_images/img1.jpg'
img = cv2.imread(path)
boxes, conf = mtcnn.detect(img)
count = 0
# If there is no confidence that in the frame is a face, don't draw a rectangle around it
if conf[0] !=  None:     
    for (x, y, w, h) in boxes:
        count += 1
        text = f"{conf[0]*100:.2f}%"
        x, y, w, h = int(x), int(y), int(w), int(h)
        cv2.putText(
                      img, #numpy array on which text is written
                      "FaceNet", #text
                      (10,50), #position at which writing has to start
                      cv2.FONT_HERSHEY_SIMPLEX, #font family
                      2, #font size
                      (250, 100, 0, 0), #font color
                      3) #font stroke
        cv2.rectangle(img, (x, y), (w, h), (250, 100, 0, 0), 2)
cv2.imwrite('/content/drive/MyDrive/YouTube-Gender-Prediction-Using-Faces-main/Data/Gender Detection/test_images/img1(facenet).jpg',img)
print(count)
# cap.release()
cv2.destroyAllWindows()
end_time = time.time()
print(f"Execution time: {end_time-start_time}") 

4
Execution time: 0.3332958221435547


##OpenCV

In [ ]:
# Import Libraries
import cv2
import numpy as np
import numpy as np
import time

start_time = time.time()
# Each Caffe Model impose the shape of the input image also image preprocessing is required like mean
# substraction to eliminate the effect of illunination changes
MODEL_MEAN_VALUES = (78.4263377603, 87.7689143744, 114.895847746)

# https://raw.githubusercontent.com/opencv/opencv/master/samples/dnn/face_detector/deploy.prototxt
FACE_PROTO = "/content/drive/MyDrive/YouTube-Gender-Prediction-Using-Faces-main/Data/Gender Detection/weights/deploy.prototxt.txt"

# https://raw.githubusercontent.com/opencv/opencv_3rdparty/dnn_samples_face_detector_20180205_fp16/res10_300x300_ssd_iter_140000_fp16.caffemodel
FACE_MODEL = "/content/drive/MyDrive/YouTube-Gender-Prediction-Using-Faces-main/Data/Gender Detection/weights/res10_300x300_ssd_iter_140000_fp16.caffemodel"

# load face Caffe model
face_net = cv2.dnn.readNetFromCaffe(FACE_PROTO, FACE_MODEL)


def get_faces(frame, confidence_threshold=0.8):
    '''Detect all the faces present in a frame'''
   
    # convert the frame into a blob to be ready for NN input
    blob = cv2.dnn.blobFromImage(frame, 1.0, (300, 300), (104, 177.0, 123.0))
  
    # set the image as input to the NN
    face_net.setInput(blob)
   
    # perform inference and get predictions
    output = np.squeeze(face_net.forward())
   
    # initialize the result list
    faces = []
    
    # Loop over the faces detected
    for i in range(output.shape[0]):
        confidence = output[i, 2]
        if confidence > confidence_threshold:
            box = output[i, 3:7] * \
                np.array([frame.shape[1], frame.shape[0],
                         frame.shape[1], frame.shape[0]])
            
            # convert to integers
            start_x, start_y, end_x, end_y = box.astype(np.int)
            
            # widen the box a little
            start_x, start_y, end_x, end_y = start_x - \
                10, start_y - 10, end_x + 10, end_y + 10
            start_x = 0 if start_x < 0 else start_x
            start_y = 0 if start_y < 0 else start_y
            end_x = 0 if end_x < 0 else end_x
            end_y = 0 if end_y < 0 else end_y
           
            # append to our list
            faces.append((start_x, start_y, end_x, end_y))
    return faces

def get_optimal_font_scale(text, width):
    '''Determine the optimal font scale based on the hosting frame width'''

    for scale in reversed(range(0, 60, 1)):
        textSize = cv2.getTextSize(text, fontFace=cv2.FONT_HERSHEY_DUPLEX, fontScale=scale/10, thickness=1)
        new_width = textSize[0][0]
        if (new_width <= width):
            return scale/10
    return 1

def gender_predict():
    '''Predict the gender in a video'''

    male_fps = 0
    female_fps = 0
    total_fps = 0
    global duration
    global fps
    global count
    global frame_count
    count = 0

    try:
        # create a new cam object
        cap = cv2.VideoCapture("/content/drive/MyDrive/YouTube-Gender-Prediction-Using-Faces-main/Data/Gender Detection/Make It Extraordinary Albert Bartlett 10 Sec TV Ad 2021.mp4")
        
        while True:
            status, frame = cap.read()
            frame = cv2.resize(frame, (600, 400))
            
            # predict the faces
            faces = get_faces(frame)
                
            
            # Loop over the faces detected
            for i, (start_x, start_y, end_x, end_y) in enumerate(faces):
                count += 1
                face_img = frame[start_y: end_y, start_x: end_x]

                blob = cv2.dnn.blobFromImage(image=face_img, scalefactor=1.0, size=(
                    227, 227), mean=MODEL_MEAN_VALUES, swapRB=False, crop=False)

                cv2.rectangle(frame, (start_x, start_y), (end_x, end_y), (255, 255, 255), 2)
                
            # Quit midway
            if cv2.waitKey(1) == ord("q"):
                break
        
        # Cleanup
        cv2.destroyAllWindows()
    except: Exception
    print(count)

if __name__ == '__main__':
    gender_predict()
    end_time = time.time()
    print(f"Execution time: {end_time-start_time}") 



127
Execution time: 13.89093279838562


In [ ]:
# Import Libraries
import cv2
import numpy as np
import numpy as np
import time

start_time = time.time()
# Each Caffe Model impose the shape of the input image also image preprocessing is required like mean
# substraction to eliminate the effect of illunination changes
MODEL_MEAN_VALUES = (78.4263377603, 87.7689143744, 114.895847746)

# https://raw.githubusercontent.com/opencv/opencv/master/samples/dnn/face_detector/deploy.prototxt
FACE_PROTO = "/content/drive/MyDrive/YouTube-Gender-Prediction-Using-Faces-main/Data/Gender Detection/weights/deploy.prototxt.txt"

# https://raw.githubusercontent.com/opencv/opencv_3rdparty/dnn_samples_face_detector_20180205_fp16/res10_300x300_ssd_iter_140000_fp16.caffemodel
FACE_MODEL = "/content/drive/MyDrive/YouTube-Gender-Prediction-Using-Faces-main/Data/Gender Detection/weights/res10_300x300_ssd_iter_140000_fp16.caffemodel"

# load face Caffe model
face_net = cv2.dnn.readNetFromCaffe(FACE_PROTO, FACE_MODEL)


def get_faces(frame, confidence_threshold=0.5):
    '''Detect all the faces present in a frame'''
   
    # convert the frame into a blob to be ready for NN input
    blob = cv2.dnn.blobFromImage(frame, 1.0, (300, 300), (104, 177.0, 123.0))
  
    # set the image as input to the NN
    face_net.setInput(blob)
   
    # perform inference and get predictions
    output = np.squeeze(face_net.forward())
   
    # initialize the result list
    faces = []
    
    # Loop over the faces detected
    for i in range(output.shape[0]):
        confidence = output[i, 2]
        if confidence > confidence_threshold:
            box = output[i, 3:7] * \
                np.array([frame.shape[1], frame.shape[0],
                         frame.shape[1], frame.shape[0]])
            
            # convert to integers
            start_x, start_y, end_x, end_y = box.astype(np.int)
            
            # widen the box a little
            start_x, start_y, end_x, end_y = start_x - \
                10, start_y - 10, end_x + 10, end_y + 10
            start_x = 0 if start_x < 0 else start_x
            start_y = 0 if start_y < 0 else start_y
            end_x = 0 if end_x < 0 else end_x
            end_y = 0 if end_y < 0 else end_y
           
            # append to our list
            faces.append((start_x, start_y, end_x, end_y))
    return faces

def get_optimal_font_scale(text, width):
    '''Determine the optimal font scale based on the hosting frame width'''

    for scale in reversed(range(0, 60, 1)):
        textSize = cv2.getTextSize(text, fontFace=cv2.FONT_HERSHEY_DUPLEX, fontScale=scale/10, thickness=1)
        new_width = textSize[0][0]
        if (new_width <= width):
            return scale/10
    return 1

def gender_predict():
    '''Predict the gender in a video'''

    male_fps = 0
    female_fps = 0
    total_fps = 0
    global duration
    global fps
    global count
    global frame_count
    count = 0

    try:
        path = '/content/drive/MyDrive/YouTube-Gender-Prediction-Using-Faces-main/Data/Gender Detection/test_images/img0.jpg'
        frame = cv2.imread(path)
        faces = get_faces(frame)
        for i, (start_x, start_y, end_x, end_y) in enumerate(faces):
            count += 1
            face_img = frame[start_y: end_y, start_x: end_x]

            blob = cv2.dnn.blobFromImage(image=face_img, scalefactor=1.0, size=(
                227, 227), mean=MODEL_MEAN_VALUES, swapRB=False, crop=False)

            cv2.rectangle(frame, (start_x, start_y), (end_x, end_y), (255, 255, 0, 255), 2)
            cv2.putText(
                      frame, #numpy array on which text is written
                      "OpenCV", #text
                      (10,50), #position at which writing has to start
                      cv2.FONT_HERSHEY_SIMPLEX, #font family
                      1, #font size
                      (255, 255, 0, 255), #font color
                      3) #font stroke
        cv2.imwrite('/content/drive/MyDrive/YouTube-Gender-Prediction-Using-Faces-main/Data/Gender Detection/test_images/img0(opencv).jpg',frame)
        # Cleanup
        cv2.destroyAllWindows()
    except: Exception
    print(count)

if __name__ == '__main__':
    gender_predict()
    end_time = time.time()
    print(f"Execution time: {end_time-start_time}") 



0
Execution time: 0.14382052421569824


##RetinaFace

In [ ]:
from retinaface import RetinaFace as rf
import cv2
import time

start_time = time.time()
 
cap = cv2.VideoCapture("/content/drive/MyDrive/YouTube-Gender-Prediction-Using-Faces-main/Data/Gender Detection/Make It Extraordinary Albert Bartlett 10 Sec TV Ad 2021.mp4")
count, face_count = 0, 0
try:
    while True:
        # read frame from video 
        status, frame = cap.read()
        frame = cv2.resize(frame, (600, 400))
        # predict the faces
        obj = rf.detect_faces(frame, threshold = 0.75)
        if type(obj) != dict:
            continue
        else:
            count += 1
            # Loop over the faces detected
            for key in obj.keys():
                face_count += 1
                identity = obj[key]
                
                face_area = identity["facial_area"]
                cv2.rectangle(frame, (face_area[2], face_area[3]), (face_area[0], face_area[1]), (255, 255, 0), 1)
            
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
except: Exception
print(face_count)
cap.release()
cv2.destroyAllWindows()
end_time = time.time()
print(f"Execution time: {end_time-start_time}") 

440
Execution time: 40.88335847854614


In [ ]:
from retinaface import RetinaFace as rf
import cv2
import time

start_time = time.time()
path = '/content/drive/MyDrive/YouTube-Gender-Prediction-Using-Faces-main/Data/Gender Detection/test_images/img2.jpg'
img = cv2.imread(path)
obj = rf.detect_faces(img, threshold = 0.75)
face_count = 0
if type(obj) == dict:
    # Loop over the faces detected
    for key in obj.keys():
        face_count += 1
        identity = obj[key]
        
        face_area = identity["facial_area"]

        cv2.putText(
                      img, #numpy array on which text is written
                      "RetinaFace", #text
                      (10,50), #position at which writing has to start
                      cv2.FONT_HERSHEY_SIMPLEX, #font family
                      2, #font size
                      (0, 255, 255), #font color
                      3) #font stroke

        cv2.rectangle(img, (face_area[2], face_area[3]), (face_area[0], face_area[1]), (0, 255, 255), 1)
cv2.imwrite('/content/drive/MyDrive/YouTube-Gender-Prediction-Using-Faces-main/Data/Gender Detection/test_images/retinaface-img2.jpg',img)  
print(face_count)
end_time = time.time()
print(f"Execution time: {end_time-start_time}") 

13
Execution time: 0.5800893306732178


##MTCNN

In [ ]:
from mtcnn import MTCNN
import cv2
import time

start_time = time.time()
#Load the haarcascade file
detector = MTCNN()
 
cap = cv2.VideoCapture("/content/drive/MyDrive/YouTube-Gender-Prediction-Using-Faces-main/Data/Gender Detection/Make It Extraordinary Albert Bartlett 10 Sec TV Ad 2021.mp4")
count = 0
try:
    while(True):
        ret, frame = cap.read()
        frame = cv2.resize(frame, (600, 400))
        boxes = detector.detect_faces(frame)
        for box in boxes:
            count += 1
            x, y, w, h = box['box']

            cv2.rectangle(frame, (x, y), (x+w, y+h), 
                                (255, 0, 0), 2)
        
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
except: Exception
print(count)
cap.release()
cv2.destroyAllWindows()
end_time = time.time()
print(f"Execution time: {end_time-start_time}") 

174
Execution time: 125.92230319976807


In [ ]:
from mtcnn import MTCNN
import cv2
import time

start_time = time.time()
#Load the haarcascade file
detector = MTCNN()

# except: Exception
path = '/content/drive/MyDrive/YouTube-Gender-Prediction-Using-Faces-main/Data/Gender Detection/test_images/img2.jpg'
img = cv2.imread(path)
count = 0
boxes = detector.detect_faces(img)
for box in boxes:
    count += 1
    x, y, w, h = box['box']
    cv2.putText(
                      img, #numpy array on which text is written
                      "MTCNN", #text
                      (10,50), #position at which writing has to start
                      cv2.FONT_HERSHEY_SIMPLEX, #font family
                      2, #font size
                      (255, 255, 0, 255), #font color
                      2) #font stroke
        
    cv2.rectangle(img, (x, y), (x+w, y+h), 
                        (255, 255, 0, 255), 1)
print(count)
cv2.imwrite('/content/drive/MyDrive/YouTube-Gender-Prediction-Using-Faces-main/Data/Gender Detection/test_images/mtcnn-img2.jpg',img)

#cap.release()
cv2.destroyAllWindows()
end_time = time.time()
print(f"Execution time: {end_time-start_time}") 

7
Execution time: 1.5873701572418213
